In [28]:
import cv2
import numpy as np
import pandas as pd
import base64
from io import BytesIO
from PIL import Image
import os
from gradio_client import Client


In [29]:
def convert_image_to_base64(image):
    _, buffer = cv2.imencode('.jpg', image)
    img_base64 = base64.b64encode(buffer).decode('utf-8')
    return img_base64

In [30]:
def get_caption_from_api(image):
    client = Client("fancyfeast/joy-caption-pre-alpha")
    result = client.predict(
		input_image=image,
		api_name="/stream_chat"
)
    return result

In [31]:
def get_video_files(directory):
    return [f for f in os.listdir(directory) if f.endswith('.mp4')]

In [32]:
def sample_frames(video_path, interval):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    data = []  # List to store frame number, video_id, and base64-encoded frame
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Sample a frame every 'interval' frames
        if frame_count % interval == 0:
            img_base64 = convert_image_to_base64(frame)  # Convert the frame to base64
            data.append({'frame_number': frame_count, 'frame_base64': img_base64})
        
        frame_count += 1
    
    cap.release()
    return data

In [33]:
def process_videos_in_directory(directory, interval=60):
    video_files = get_video_files(directory)
    
    all_data = []
    
    for video_file in video_files:
        video_path = os.path.join(directory, video_file)
        video_id = os.path.splitext(video_file)[0]  # Extract video_id from filename
        
        # Sample frames from the video
        sampled_frames = sample_frames(video_path, interval)
        
        # Generate captions for each sampled frame
        for frame_data in sampled_frames:
            caption = get_caption_from_api(frame_data['frame_base64'])  # Replace with actual captioning
            frame_data['video_id'] = video_id
            frame_data['caption'] = caption
            all_data.append(frame_data)
    
    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(all_data)
    return df

In [ ]:
directory_path = r"/Users/yicheng/Downloads/test_video"
df = process_videos_in_directory(directory_path, 60)